In [1]:
from bs4 import BeautifulSoup  #beatuisoup 解析html
import requests                 #获取html
import pandas as pd
import numpy as np

In [ ]:
#伪造agent
user_agents=[
    "Mozilla/5.0 (Linux; U; Android 2.3.6; en-us; Nexus S Build/GRK39F) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
	"Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.5 (KHTML, like Gecko) Chrome/4.0.249.0 Safari/532.5",
	"Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/532.9 (KHTML, like Gecko) Chrome/5.0.310.0 Safari/532.9",
	"Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.7 (KHTML, like Gecko) Chrome/7.0.514.0 Safari/534.7",
	"Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/9.0.601.0 Safari/534.14",
	"Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/10.0.601.0 Safari/534.14",
	"Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.20 (KHTML, like Gecko) Chrome/11.0.672.2 Safari/534.20",
	"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.27 (KHTML, like Gecko) Chrome/12.0.712.0 Safari/534.27",
	"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.24 Safari/535.1",
	"Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.2 (KHTML, like Gecko) Chrome/15.0.874.120 Safari/535.2",
	"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.36 Safari/535.7",
	"Mozilla/5.0 (Windows; U; Windows NT 6.0 x64; en-US; rv:1.9pre) Gecko/2008072421 Minefield/3.0.2pre",
	"Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.10) Gecko/2009042316 Firefox/3.0.10",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36 Edg/118.0.2088.61"
    ]

In [3]:
#构建ip池，绕过反爬
import random
api_url = "https://tps.kdlapi.com/api/gettps/?secret_id=-------&num=1&signature=--------&pt=1&sep=1"
#此处采用快代理的api接口，获取稳定的可用的ip
proxy_ip = requests.get(api_url).text

def getHtml(target_url):
    username = '用户名'
    password = '用户密码'
    proxies = {
    "http": "http://%(user)s:%(pwd)s@%(proxy)s/" % {"user": username, "pwd": password, "proxy": proxy_ip},
    "https": "http://%(user)s:%(pwd)s@%(proxy)s/" % {"user": username, "pwd": password, "proxy": proxy_ip}
    }#设置代理ip 

    while(True):
        user_agent = random.choice(user_agents)
        headers = {
            'user-agent': user_agent
        }
        response = requests.get(url = target_url,headers=headers, proxies=proxies)

        if response.status_code == 200:  #200 OK 成功获取网页
            res = BeautifulSoup(response.text,'lxml')
            if("中富电路" in res.find("title").text):  #如果出现中富，说明被封ip重定向
                continue
            return res
        else:
            continue


In [91]:
res = getHtml("https://guba.eastmoney.com/list,200011.html")

In [4]:
def getPage(base_url,i):#获取某一页的帖子
    All_data = []
    url = base_url + str(i) + '.html' #拼接出网页地址
    html = getHtml(url)                 #此处的html是函数中return的解析过的html
    post_list=html.find_all('tr',class_ = 'listitem')
    if post_list == []:             #得到的是空列表，当前页没有内容
        return None
    for post in post_list:
        read_counts=post.find('div',class_='read').text       #获取帖子阅读数
        comment_counts=post.find('div',class_='reply').text #获取帖子评论数
        title=post.find('div',class_='title').text             #获取帖子标题    
        time=post.find('div',class_='update').text              #获取更新时间
        data = [read_counts,comment_counts,title,time]
        All_data.append(data)
    return All_data

In [96]:
result = []
for i in range(1,1477): #用于测试600000的爬取
    base_url = 'https://guba.eastmoney.com/list,600000_'
    unit_result = getPage(base_url,i)
    result.extend(unit_result)

In [97]:
df = pd.DataFrame(result,columns=['read','comment','title','time'])

In [98]:
df.tail()

,read,comment,title,time
117900,128,0,上海浦东发展银行股份有限公司第一届董事会第九次会议补充公告,08-08 12:00
117901,384,0,上海浦东发展银行股份有限公司召开2001年第一次临时股东大会公告,08-05 12:00
117902,140,0,上海浦东发展银行股份有限公司2001年中期报告,06-30 12:00
117903,361,0,上海浦东发展银行股份有限公司公告,04-06 12:00
117904,109,0,上海市联合律师事务所关于上海浦东发展银行股份有限公司2000年度,11-23 12:00


In [114]:
df.to_csv('./stock_a/600000.csv')

接下来一段是对各个股票的爬取，耗时太大，不建议运行

In [15]:
stock_a = pd.read_csv('./volume.CSV',low_memory=False,encoding = 'ANSI')
code_list = []
for item in stock_a.columns[1:]:
    code_list.append(item.split('.')[0]) #获取股票代码

In [17]:
code_list = code_list[1:]

In [18]:
len(code_list)

4757

In [21]:
def scrapter(code):
    base_url = 'https://guba.eastmoney.com/list,'+code+"_"
    if code == '300814': #先跳过重定向页面
        return None
    result = []
    for i in range(1,10000):
        unit_result = getPage(base_url,i)
        if(unit_result == None):
            break #reach the end
        result.extend(unit_result)
    
    df = pd.DataFrame(result,columns=['read','comment','title','time'])
    path = './stock_a/'+code+'.csv'
    df.to_csv(path)

In [14]:
for code in code_list:
    base_url = 'https://guba.eastmoney.com/list,'+code+"_"
    if code == '300814': #先跳过重定向页面
        continue
    result = []
    for i in range(1,10000):
        unit_result = getPage(base_url,i)
        if(unit_result == None):
            break #reach the end
        result.extend(unit_result)
    
    df = pd.DataFrame(result,columns=['read','comment','title','time'])
    path = './stock_b/'+code+'.csv'
    df.to_csv(path)

运行到这